## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_weather_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.23,86,100,25.66,light rain
1,1,Srednekolymsk,RU,67.4500,153.6833,64.29,26,35,12.68,scattered clouds
2,2,Albany,US,42.6001,-73.9662,48.15,83,100,0.38,overcast clouds
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,75.07,84,45,4.34,scattered clouds
4,4,Grindavik,IS,63.8424,-22.4338,48.15,100,90,28.83,heavy intensity rain


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_weather_df.loc[(city_data_weather_df["Max Temp"] <= max_temp) & \
                                       (city_data_weather_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.23,86,100,25.66,light rain
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,75.07,84,45,4.34,scattered clouds
11,11,Butaritari,KI,3.0707,172.7902,83.19,78,91,15.41,overcast clouds
15,15,Nabire,ID,-3.3667,135.4833,88.12,63,100,4.61,overcast clouds
23,23,Bambous Virieux,MU,-20.3428,57.7575,78.22,94,75,9.17,broken clouds
27,27,Hilo,US,19.7297,-155.0900,79.29,83,90,10.36,overcast clouds
28,28,The Valley,AI,18.2170,-63.0578,80.46,72,22,19.33,few clouds
34,34,Touros,BR,-5.1989,-35.4608,76.10,86,44,11.81,scattered clouds
35,35,Mahebourg,MU,-20.4081,57.7000,78.26,94,75,9.22,broken clouds
38,38,Bengkulu,ID,-3.8004,102.2655,84.00,67,99,4.72,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
preferred_cities_df.count()

City_ID                446
City                   446
Country                446
Lat                    446
Lng                    446
Max Temp               446
Humidity               446
Cloudiness             446
Wind Speed             446
Current Description    446
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_df = preferred_cities_df.dropna()
new_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.23,light rain,-23.1203,-134.9692,
3,Arraial Do Cabo,BR,75.07,scattered clouds,-22.9661,-42.0278,
11,Butaritari,KI,83.19,overcast clouds,3.0707,172.7902,
15,Nabire,ID,88.12,overcast clouds,-3.3667,135.4833,
23,Bambous Virieux,MU,78.22,broken clouds,-20.3428,57.7575,
27,Hilo,US,79.29,overcast clouds,19.7297,-155.0900,
28,The Valley,AI,80.46,few clouds,18.2170,-63.0578,
34,Touros,BR,76.10,scattered clouds,-5.1989,-35.4608,
35,Mahebourg,MU,78.26,broken clouds,-20.4081,57.7000,
38,Bengkulu,ID,84.00,overcast clouds,-3.8004,102.2655,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.isnull().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             38
dtype: int64

In [25]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.23,light rain,-23.1203,-134.9692,Pension Maro'i
3,Arraial Do Cabo,BR,75.07,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
11,Butaritari,KI,83.19,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
15,Nabire,ID,88.12,overcast clouds,-3.3667,135.4833,Hotel Nusantara Nabire
23,Bambous Virieux,MU,78.22,broken clouds,-20.3428,57.7575,Casa Tia Villa
27,Hilo,US,79.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
28,The Valley,AI,80.46,few clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
34,Touros,BR,76.10,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
35,Mahebourg,MU,78.26,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
38,Bengkulu,ID,84.00,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig 

Figure(layout=FigureLayout(height='420px'))